### **Data Gathering**

Housing API

In [ ]:
import requests
import http.client, urllib.parse
import csv

# HOUSING API
def getHouses(csv_, cities):

    csvfile = open(csv_, 'w', newline='')
    writer = csv.DictWriter(csvfile, fieldnames=['address', 'price', 'city', 'squareFootage', 'latitude', 'longitude'])

    url = "https://realty-mole-property-api.p.rapidapi.com/saleListings"

    for city, state in cities:
        querystring = {"city":city,"state":state,"limit":"500"}

        headers = {
            "X-RapidAPI-Key": "43686bd243mshe8a0be6f9e0556cp10a1bbjsn58033bf0f546",
            "X-RapidAPI-Host": "realty-mole-property-api.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)

        for house in response.json():
            if type(house) == dict:
                address = house.get('formattedAddress')
                price = house.get('price')
                city = house.get('city')
                sqf = house.get('squareFootage')
                latitude = house.get('latitude')
                longitude = house.get('longitude')
                writer.writerow({'address':address, 'price':price, 'city':city, 'squareFootage':sqf, 
                                 'latitude':latitude, 'longitude':longitude})

    csvfile.close()  

In [ ]:
#RUNNING THIS WILL DUPLICATE ENTRIES ON CURRENT FILE
cities = [('Los Angeles', 'CA'), ('Anaheim','CA'), ('Long Beach', 'CA'), ('Chicago', 'IL'), ('Naperville', 'IL'), ('Elgin', 'IL'),
             ('Dallas', 'TX'), ('Fort Worth', 'TX'), ('Arlington', 'TX'), ('Washington', 'DC'), ('Arlington', 'VA'), ('Alexandria', 'VA')]

getHouses('houses.csv', cities)

Yellow Pages API

In [ ]:
import requests
import time
import pandas as pd
import lxml.html as lx
from bs4 import BeautifulSoup

In [ ]:
def getPlaces(city, state, term):
    addresses = list()
    for i in range(1, 6):
        if i == 1:
            url = "https://www.yellowpages.com/search?search_terms=" + str(term) + "&geo_location_terms=" + str(city) + "%2C+" + str(state)
        else:
            url = "https://www.yellowpages.com/search?search_terms=" + str(term) + "&geo_location_terms=" + str(city) + "%2C+" + str(state) + "&page=" + str(i)
        time.sleep(0.05)
        response = requests.get(url,
                            headers = {"accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
                                        "accept-encoding": "gzip, deflate, br",
                                        "accept-language": "en-US,en;q=0.9",
                                        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"})

        html = response.text
        page = BeautifulSoup(html, "html.parser")
        address = str(page.find_all("div", class_="adr")).split('<div class="adr"><div class="street-address">') 

        for j in range(1, len(address)):
            if address[j] not in addresses:
                addresses.append(address[j].replace('</div><div class="locality">', ",").replace('</div></div>, ', "").replace("</div></div>]", "").replace('<div class="adr"><div class="locality">', ""))
            else:
                continue
    return pd.DataFrame(pd.Series(addresses).unique(), columns = ["Address"])
    

In [ ]:
places = ['hospital', 'grocery', "gym", "parks", "beaches", "cemetary", "shopping", "resturant", "golf"]

LosAngeles = pd.DataFrame()
Anaheim = pd.DataFrame()
LongBeach = pd.DataFrame()
Chicago = pd.DataFrame()
Naperville = pd.DataFrame()
Elgin = pd.DataFrame()
Dallas = pd.DataFrame()
FortWorth = pd.DataFrame()
ArlingtonTX = pd.DataFrame()
Washington = pd.DataFrame()
ArlingtonVA = pd.DataFrame()
Alexandria = pd.DataFrame()

for p in range(0, len(places)):
    time.sleep(0.05)
    name = places[p]
    LosAngeles[name] = getPlaces('Los+Angeles', 'CA', places[p])
    Anaheim[name] = getPlaces('Anaheim', 'CA', places[p])
    LongBeach[name] = getPlaces('Long+Beach', 'CA', places[p])
    Chicago[name] = getPlaces('Chicago', 'IL', places[p])
    Naperville[name] = getPlaces('Naperville', 'IL', places[p])
    Elgin[name] = getPlaces('Elgin', 'IL', places[p])
    Dallas[name] = getPlaces('Dallas', 'TX', places[p])
    FortWorth[name] = getPlaces('Fort+Worth', 'TX', places[p])
    ArlingtonTX[name] = getPlaces('Arlington', 'TX', places[p])
    Washington[name] = getPlaces('Washington', 'DC', places[p])
    ArlingtonVA[name] = getPlaces('Arlington', 'VA', places[p])
    Alexandria[name] = getPlaces('Alexandria', 'VA', places[p])

In [ ]:
thingsToDo = pd.concat([LosAngeles, Anaheim, LongBeach, 
                   Chicago, Naperville, Elgin, 
                   Dallas,FortWorth, ArlingtonTX, 
                   Washington, ArlingtonVA, Alexandria])

Common Core Data Scraping

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [ ]:
#turn data into address
def address(x):
    x = x.iloc[:,range(9,13)]
    x['address'] = x[x.columns[:]].apply(
        lambda x: ', '.join(x.dropna().astype(str)),
        axis=1
    )
    x = pd.DataFrame(x[x.columns[4]])
    return x

In [ ]:
#Read data in
#La
la = pd.read_excel("./data/la.xlsx")
la = address(la)
long_beach = pd.read_excel("./data/long_beach.xlsx")
long_beach = address(long_beach)
anaheim = pd.read_excel("./data/anaheim.xlsx")
anaheim = address(anaheim)

#Chicago
chicago = pd.read_excel("./data/chicago.xlsx")
chicago = address(chicago)
naperville = pd.read_excel("./data/naperville.xlsx")
naperville = address(naperville)
elgin = pd.read_excel("./data/elgin.xlsx")
elgin = address(elgin)

#Dallas
dallas = pd.read_excel("./data/dallas.xlsx")
dallas = address(dallas)
fort_worth = pd.read_excel("./data/fort_worth.xlsx")
fort_worth = address(fort_worth)
arlington = pd.read_excel("./data/arlington.xlsx")
arlington = address(arlington)

#Washington
washington = pd.read_excel("./data/dc.xlsx")
washington = address(washington)
arlington_va = pd.read_excel("./data/arlington_va.xlsx")
arlington_va = address(arlington_va)
alexandria = pd.read_excel("./data/alexandria.xlsx")
alexandria = address(alexandria)

### **Data Manipulation**

In [1]:
from sklearn import preprocessing
import http.client, urllib.parse
import pandas as pd
import numpy as np
import math
import json
from geopy.distance import geodesic
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def getDist(houses, locations):
    distances = []
        
    for ind in houses.index:
        y1 = houses['Longitude'][ind]
        x1 = houses['Latitude'][ind]
        coord1 = (y1,x1)
        min_dist = 9999999
        for i in locations.index:
            y2 = locations['Latitude'][i]
            x2 = locations['Longitude'][i]
            coord2 = (y2,x2)
            try:
                dist = geodesic(coord1, coord2).miles
                if dist < min_dist:
                    min_dist = dist
            except:
                pass
        distances.append(min_dist)  

        
    return distances

def getCoords(df):
    
    #api info
    conn = http.client.HTTPConnection('api.positionstack.com')
    key = 'df3b33e27a0f6451fd9aae993c6a26fa'
    
    
    def query(coord, address):

        params = urllib.parse.urlencode({
            'access_key': key,
            'query': address,
            'limit': 1
            })
        
        conn.request('GET', '/v1/forward?{}'.format(params))
        results = conn.getresponse()
        data = json.loads(results.read())['data'][0] #converts json to dict
        
        if coord == 'x':
            result = data['latitude']
        else:
            result = data['longitude']
            
            return result
    

    lat = [query('x', address) for address in df.iloc[:,0]]
    lon = [query('y', address) for address in df.iloc[:,0]]
    
    df['latitude'] = lat
    df['longitude'] = lon
    
    return df

In [4]:
#IMPORT AND ORGANIZE DATA

houses = pd.read_csv('../data/final/houses_clean.csv')

houses_la = houses[houses['City'] == 'Los Angeles']
houses_anaheim = houses[houses['City'] == 'Anaheim']
houses_longbeach = houses[houses['City'] == 'Long Beach']

houses_dallas = houses[houses['City'] == 'Dallas']
houses_fortworth = houses[houses['City'] == 'Fort Worth']
houses_arlington_tx = houses[houses['City'] == 'ArlingtonTX']

houses_chicago = houses[houses['City'] == 'Chicago']
houses_naperville = houses[houses['City'] == 'Naperville']
houses_elgin = houses[houses['City'] == 'Elgin']

houses_washington = houses[houses['City'] == 'Washington']
houses_alexandria = houses[houses['City'] == 'Alexandria']
houses_arlington_va = houses[houses['City'] == 'ArlingtonVA']


schools = pd.read_csv('../data/final/schools.csv')
schools_la = schools[schools['City'] == 'Los Angeles']
schools_anaheim = schools[schools['City'] == 'Anaheim']
schools_longbeach = schools[schools['City'] == 'Long Beach']

schools_dallas = schools[schools['City'] == 'Dallas']
schools_fortworth = schools[schools['City'] == 'Fort Worth']
schools_atx = schools[schools['City'] == 'ArlingtonTX']

schools_chicago = schools[schools['City'] == 'Chicago']
schools_naperville = schools[schools['City'] == 'Naperville']
schools_elgin = schools[schools['City'] == 'Elgin']

schools_washington = schools[schools['City'] == 'Washington']
schools_alex = schools[schools['City'] == 'Alexandria']
schools_ava = schools[schools['City'] == 'ArlingtonVA']


hospitals = pd.read_csv('../data/final/hospital.csv')

hospitals_la = hospitals[hospitals['City'] == 'Los Angeles']
hospitals_a = hospitals[hospitals['City'] == 'Anaheim']
hospitals_lb = hospitals[hospitals['City'] == 'Long Beach']

hospitals_d = hospitals[hospitals['City'] == 'Dallas']
hospitals_fw = hospitals[hospitals['City'] == 'Fort Worth']
hospitals_atx = hospitals[hospitals['City'] == 'ArlingtonTX']

hospitals_c = hospitals[hospitals['City'] == 'Chicago']
hospitals_n = hospitals[hospitals['City'] == 'Naperville']
hospitals_e = hospitals[hospitals['City'] == 'Elgin']

hospitals_dc = schools[schools['City'] == 'Washington']
hospitals_alex = schools[schools['City'] == 'Alexandria']
hospitals_ava = schools[schools['City'] == 'ArlingtonVA']

grocery = pd.read_csv('../data/final/grocery.csv')

grocery_la = grocery[grocery['City'] == 'Los Angeles']
grocery_a = grocery[grocery['City'] == 'Anaheim']
grocery_lb = grocery[grocery['City'] == 'Long Beach']

grocery_d = grocery[grocery['City'] == 'Dallas']
grocery_fw = grocery[grocery['City'] == 'Fort Worth']
grocery_atx = grocery[grocery['City'] == 'ArlingtonTX']

grocery_c = grocery[grocery['City'] == 'Chicago']
grocery_n = grocery[grocery['City'] == 'Naperville']
grocery_e = grocery[grocery['City'] == 'Elgin']

grocery_dc = grocery[grocery['City'] == 'Washington']
grocery_alex = grocery[grocery['City'] == 'Alexandria']
grocery_ava = grocery[grocery['City'] == 'ArlingtonVA']

gym = pd.read_csv('../data/final/gym.csv')

gym_la = gym[gym['City'] == 'Los Angeles']
gym_lb = gym[gym['City'] == 'Long Beach']
gym_a = gym[gym['City'] == 'Anaheim']

gym_d = gym[gym['City'] == 'Dallas']
gym_atx = gym[gym['City'] == 'ArlingtonTX']
gym_fw = gym[gym['City'] == 'Fort Worth']

gym_c = gym[gym['City'] == 'Chicago']
gym_e = gym[gym['City'] == 'Elgin']
gym_n = gym[gym['City'] == 'Naperville']

gym_dc = gym[gym['City'] == 'Washington']
gym_ava = gym[gym['City'] == 'ArlingtonVA']
gym_alex = gym[gym['City'] == 'Alexandria']

parks = pd.read_csv('../data/final/parks.csv')

parks_la = parks[parks['City'] == 'Los Angeles']
parks_lb = parks[parks['City'] == 'Long Beach']
parks_a = parks[parks['City'] == 'Anaheim']

parks_d = parks[parks['City'] == 'Dallas']
parks_atx = parks[parks['City'] == 'ArlingtonTX']
parks_fw = parks[parks['City'] == 'Fort Worth']

parks_c = parks[parks['City'] == 'Chicago']
parks_e = parks[parks['City'] == 'Elgin']
parks_n = parks[parks['City'] == 'Naperville']

parks_dc = parks[parks['City'] == 'Washington']
parks_ava = parks[parks['City'] == 'ArlingtonVA']
parks_alex = parks[parks['City'] == 'Alexandria']

beaches = pd.read_csv('../data/final/beach.csv')

beaches_la = beaches[beaches['City'] == 'Los Angeles']
beaches_lb = beaches[beaches['City'] == 'Long Beach']
beaches_a = beaches[beaches['City'] == 'Anaheim']

beaches_d = beaches[beaches['City'] == 'Dallas']
beaches_atx = beaches[beaches['City'] == 'ArlingtonTX']
beaches_fw = beaches[beaches['City'] == 'Fort Worth']

beaches_c = beaches[beaches['City'] == 'Chicago']
beaches_e = beaches[beaches['City'] == 'Elgin']
beaches_n = beaches[beaches['City'] == 'Naperville']

beaches_dc = beaches[beaches['City'] == 'Washington']
beaches_ava = beaches[beaches['City'] == 'ArlingtonVA']
beaches_alex = beaches[beaches['City'] == 'Alexandria']

cemetary = pd.read_csv('../data/final/cemetary.csv')

cemetary_la = cemetary[cemetary['City'] == 'Los Angeles']
cemetary_lb = cemetary[cemetary['City'] == 'Long Beach']
cemetary_a = cemetary[cemetary['City'] == 'Anaheim']

cemetary_d = cemetary[cemetary['City'] == 'Dallas']
cemetary_atx = cemetary[cemetary['City'] == 'ArlingtonTX']
cemetary_fw = cemetary[cemetary['City'] == 'Fort Worth']

cemetary_c = cemetary[cemetary['City'] == 'Chicago']
cemetary_e = cemetary[cemetary['City'] == 'Elgin']
cemetary_n = cemetary[cemetary['City'] == 'Naperville']

cemetary_dc = cemetary[cemetary['City'] == 'Washington']
cemetary_ava = cemetary[cemetary['City'] == 'ArlingtonVA']
cemetary_alex = cemetary[cemetary['City'] == 'Alexandria']

shopping = pd.read_csv('../data/final/shopping.csv')

shopping_la = shopping[shopping['City'] == 'Los Angeles']
shopping_lb = shopping[shopping['City'] == 'Long Beach']
shopping_a = shopping[shopping['City'] == 'Anaheim']

shopping_d = shopping[shopping['City'] == 'Dallas']
shopping_atx = shopping[shopping['City'] == 'ArlingtonTX']
shopping_fw = shopping[shopping['City'] == 'Fort Worth']

shopping_c = shopping[shopping['City'] == 'Chicago']
shopping_e = shopping[shopping['City'] == 'Elgin']
shopping_n = shopping[shopping['City'] == 'Naperville']

shopping_dc = shopping[shopping['City'] == 'Washington']
shopping_ava = shopping[shopping['City'] == 'ArlingtonVA']
shopping_alex = shopping[shopping['City'] == 'Alexandria']

resturant = pd.read_csv('../data/final/restaurants.csv')

resturant_la = resturant[resturant['City'] == 'Los Angeles']
resturant_lb = resturant[resturant['City'] == 'Long Beach']
resturant_a = resturant[resturant['City'] == 'Anaheim']

resturant_d = resturant[resturant['City'] == 'Dallas']
resturant_atx = resturant[resturant['City'] == 'ArlingtonTX']
resturant_fw = resturant[resturant['City'] == 'Fort Worth']

resturant_c = resturant[resturant['City'] == 'Chicago']
resturant_e = resturant[resturant['City'] == 'Elgin']
resturant_n = resturant[resturant['City'] == 'Naperville']

resturant_dc = resturant[resturant['City'] == 'Washington']
resturant_ava = resturant[resturant['City'] == 'ArlingtonVA']
resturant_alex = resturant[resturant['City'] == 'Alexandria']

golf = pd.read_csv('../data/final/golf.csv')

golf_la = golf[golf['City'] == 'Los Angeles']
golf_lb = golf[golf['City'] == 'Long Beach']
golf_a = golf[golf['City'] == 'Anaheim']

golf_d = golf[golf['City'] == 'Dallas']
golf_atx = golf[golf['City'] == 'ArlingtonTX']
golf_fw = golf[golf['City'] == 'Fort Worth']

golf_c = golf[golf['City'] == 'Chicago']
golf_e = golf[golf['City'] == 'Elgin']
golf_n = golf[golf['City'] == 'Naperville']

golf_dc = golf[golf['City'] == 'Washington']
golf_ava = golf[golf['City'] == 'ArlingtonVA']
golf_alex = golf[golf['City'] == 'Alexandria']


In [5]:
#EDA

houses = pd.read_csv('../data/houses_clean.csv')

houses_la = houses.loc[houses['City'] == 'Anaheim']
houses_anaheim = houses.loc[houses['City'] == 'Los Angeles']
houses_longbeach = houses.loc[houses['City'] == 'Long Beach']

houses_dallas = houses.loc[houses['City'] == 'Dallas']
houses_fortworth = houses.loc[houses['City'] == 'Fort Worth']
houses_arlington_tx = houses.loc[houses['City'] == 'ArlingtonTX']

houses_chicago = houses.loc[houses['City'] == 'Chicago']
houses_naperville = houses.loc[houses['City'] == 'Naperville']
houses_elgin = houses.loc[houses['City'] == 'Elgin']

houses_washington = houses.loc[houses['City'] == 'Washington']
houses_alexandria = houses.loc[houses['City'] == 'Alexandria']
houses_arlington_va = houses.loc[houses['City'] == 'ArlingtonVA']

#combine cities by state
houses_ca = pd.concat([houses_la, houses_anaheim, houses_longbeach])
houses_tx = pd.concat([houses_dallas, houses_fortworth, houses_arlington_tx])
houses_dc = pd.concat([houses_washington, houses_alexandria, houses_arlington_va])
houses_il = pd.concat([houses_chicago, houses_naperville, houses_elgin])

#divide price by sq ft
houses_ca['Housing Price/SQ Ft'] = houses_ca['Housing Price']/houses_ca['SQ Ft']
houses_tx['Housing Price/SQ Ft'] = houses_tx['Housing Price']/houses_tx['SQ Ft']
houses_dc['Housing Price/SQ Ft'] = houses_dc['Housing Price']/houses_dc['SQ Ft']
houses_il['Housing Price/SQ Ft'] = houses_il['Housing Price']/houses_il['SQ Ft']

#houses_ca['log'] = np.log(houses_ca['Housing Price/SQ Ft'])
#houses_ca['log2'] = np.log2(houses_ca['Housing Price/SQ Ft'])
#houses_ca['log10'] = np.log10(houses_ca['Housing Price/SQ Ft'])

In [ ]:
houses_tx.sort_values(by = 'Housing Price/SQ Ft', ascending=True).head()
houses_il.sort_values(by = 'Housing Price/SQ Ft', ascending=False).head()
houses_ca.shape

In [ ]:
plt.hist(houses_il['Housing Price/SQ Ft'])
plt.show()

In [ ]:
plt.boxplot(houses_il['Housing Price/SQ Ft'])
plt.show()

In [ ]:
#normalize
houses_ca['Norm Price'] = preprocessing.MinMaxScaler().fit_transform(np.array(houses_ca['Housing Price/SQ Ft']).reshape(-1,1))
houses_tx['Norm Price'] = preprocessing.MinMaxScaler().fit_transform(np.array(houses_tx['Housing Price/SQ Ft']).reshape(-1,1))
houses_dc['Norm Price'] = preprocessing.MinMaxScaler().fit_transform(np.array(houses_dc['Housing Price/SQ Ft']).reshape(-1,1))
houses_il['Norm Price'] = preprocessing.MinMaxScaler().fit_transform(np.array(houses_il['Housing Price/SQ Ft']).reshape(-1,1))

In [6]:
#Initialize houses for pricing
houses = pd.read_csv('../data/houses_clean.csv')

houses_la = houses.loc[houses['City'] == 'Anaheim']
houses_anaheim = houses.loc[houses['City'] == 'Los Angeles']
houses_longbeach = houses.loc[houses['City'] == 'Long Beach']

houses_dallas = houses.loc[houses['City'] == 'Dallas']
houses_fortworth = houses.loc[houses['City'] == 'Fort Worth']
houses_arlington_tx = houses.loc[houses['City'] == 'ArlingtonTX']

houses_chicago = houses.loc[houses['City'] == 'Chicago']
houses_naperville = houses.loc[houses['City'] == 'Naperville']
houses_elgin = houses.loc[houses['City'] == 'Elgin']

houses_washington = houses.loc[houses['City'] == 'Washington']
houses_alexandria = houses.loc[houses['City'] == 'Alexandria']
houses_arlington_va = houses.loc[houses['City'] == 'ArlingtonVA']

#combine cities by state
houses_ca = pd.concat([houses_la, houses_anaheim, houses_longbeach])
houses_tx = pd.concat([houses_dallas, houses_fortworth, houses_arlington_tx])
houses_dc = pd.concat([houses_washington, houses_alexandria, houses_arlington_va])
houses_il = pd.concat([houses_chicago, houses_naperville, houses_elgin])

#divide price by sq ft
houses_ca['Housing Price/SQ Ft'] = houses_ca['Housing Price']/houses_ca['SQ Ft']
houses_tx['Housing Price/SQ Ft'] = houses_tx['Housing Price']/houses_tx['SQ Ft']
houses_dc['Housing Price/SQ Ft'] = houses_dc['Housing Price']/houses_dc['SQ Ft']
houses_il['Housing Price/SQ Ft'] = houses_il['Housing Price']/houses_il['SQ Ft']

#clean data
houses_ca = houses_ca.drop(houses_ca[houses_ca['Housing Price/SQ Ft'] > 3000].index)
houses_dc = houses_dc.drop(houses_dc[houses_dc['Housing Price/SQ Ft'] > 3000].index)

#add column
#houses_ca['Housing Price/SQ Ft'] = pd.DataFrame(houses_ca['Housing Price/SQ Ft'])
#houses_tx['Housing Price/SQ Ft'] = pd.DataFrame(houses_tx['Housing Price/SQ Ft'])
#houses_dc['Housing Price/SQ Ft'] = pd.DataFrame(houses_dc['Housing Price/SQ Ft'])
#houses_il['Housing Price/SQ Ft'] = pd.DataFrame(houses_il['Housing Price/SQ Ft'])

#normalize
houses_ca['Norm Price'] = preprocessing.MinMaxScaler().fit_transform(np.array(houses_ca['Housing Price/SQ Ft']).reshape(-1,1))
houses_tx['Norm Price'] = preprocessing.MinMaxScaler().fit_transform(np.array(houses_tx['Housing Price/SQ Ft']).reshape(-1,1))
houses_dc['Norm Price'] = preprocessing.MinMaxScaler().fit_transform(np.array(houses_dc['Housing Price/SQ Ft']).reshape(-1,1))
houses_il['Norm Price'] = preprocessing.MinMaxScaler().fit_transform(np.array(houses_il['Housing Price/SQ Ft']).reshape(-1,1))

#subset so that all city dataframes have the normalized price values
houses_la = houses_ca.loc[houses_ca['City'] == 'Anaheim']
houses_anaheim = houses_ca.loc[houses_ca['City'] == 'Los Angeles']
houses_longbeach = houses_ca.loc[houses_ca['City'] == 'Long Beach']

houses_dallas = houses_tx.loc[houses_tx['City'] == 'Dallas']
houses_fortworth = houses_tx.loc[houses_tx['City'] == 'Fort Worth']
houses_arlington_tx = houses_tx.loc[houses_tx['City'] == 'ArlingtonTX']

houses_chicago = houses_il.loc[houses_il['City'] == 'Chicago']
houses_naperville = houses_il.loc[houses_il['City'] == 'Naperville']
houses_elgin = houses_il.loc[houses_il['City'] == 'Elgin']

houses_washington = houses_dc.loc[houses_dc['City'] == 'Washington']
houses_alexandria = houses_dc.loc[houses_dc['City'] == 'Alexandria']
houses_arlington_va = houses_dc.loc[houses_dc['City'] == 'ArlingtonVA']

In [140]:
# LA DATA
hospitals_ca = pd.concat([hospitals_la, hospitals_lb, hospitals_a])
gym_ca = pd.concat([gym_la, gym_lb, gym_a])
cemetary_ca = pd.concat([cemetary_la, cemetary_lb, cemetary_a])
parks_ca = pd.concat([parks_la, parks_lb, parks_la])
beaches_ca = pd.concat([beaches_la, beaches_lb, beaches_a])
shopping_ca = pd.concat([shopping_la, shopping_lb, shopping_a])
grocery_ca = pd.concat([grocery_la, grocery_lb, grocery_a])
resturant_ca = pd.concat([resturant_la, resturant_lb, resturant_a])
golf_ca = pd.concat([golf_la, golf_lb, golf_a])
school_ca = pd.concat([schools_la, schools_longbeach, schools_anaheim])

houses_ca['dist_hospitals'] = getDist(houses_ca, hospitals_ca)
houses_ca['dist_gym'] = getDist(houses_ca, gym_ca)
houses_ca['dist_cemetary'] = getDist(houses_ca, cemetary_ca)
houses_ca['dist_parks'] = getDist(houses_ca, parks_ca)
houses_ca['dist_beaches'] = getDist(houses_ca, beaches_ca)
houses_ca['dist_shopping'] = getDist(houses_ca, shopping_ca)
houses_ca['dist_grocery'] = getDist(houses_ca, grocery_ca)
houses_ca['dist_resturant'] = getDist(houses_ca, resturant_ca)
houses_ca['dist_golf'] = getDist(houses_ca, golf_ca)
houses_ca['dist_school'] = getDist(houses_ca, school_ca)

houses_ca = houses_ca.iloc[: , 1:]

In [18]:
# CHICAGO DATA
hospitals_il = pd.concat([hospitals_c, hospitals_e, hospitals_n])
gym_il = pd.concat([gym_c, gym_e, gym_n])
cemetary_il = pd.concat([cemetary_c, cemetary_e, cemetary_n])
parks_il = pd.concat([parks_c, parks_e, parks_n])
beaches_il = pd.concat([beaches_c, beaches_e, beaches_n])
shopping_il = pd.concat([shopping_c, shopping_e, shopping_n])
grocery_il = pd.concat([grocery_c, grocery_e, grocery_n])
resturant_il = pd.concat([resturant_c, resturant_e, resturant_n])
golf_il = pd.concat([golf_c, golf_e, golf_n])
school_il = pd.concat([schools_chicago, schools_elgin, schools_naperville])


houses_il['dist_hospitals'] = getDist(houses_il, hospitals_il)
houses_il['dist_gym'] = getDist(houses_il, gym_il)
houses_il['dist_cemetary'] = getDist(houses_il, cemetary_il)
houses_il['dist_parks'] = getDist(houses_il, parks_il)
houses_il['dist_beaches'] = getDist(houses_il, beaches_il)
houses_il['dist_shopping'] = getDist(houses_il, shopping_il)
houses_il['dist_grocery'] = getDist(houses_il, grocery_il)
houses_il['dist_resturant'] = getDist(houses_il, resturant_il)
houses_il['dist_golf'] = getDist(houses_il, golf_il)
houses_il['dist_school'] = getDist(houses_il, school_il)

houses_il = houses_il.iloc[: , 1:]


In [19]:
#dallas - texas
hospitals_tx = pd.concat([hospitals_d, hospitals_fw, hospitals_atx])
gym_tx = pd.concat([gym_d, gym_fw, gym_atx])
cemetary_tx = pd.concat([cemetary_d, cemetary_fw, cemetary_atx])
parks_tx = pd.concat([parks_d, parks_fw, parks_atx])
beaches_tx = pd.concat([beaches_d, beaches_fw, beaches_atx])
shopping_tx = pd.concat([shopping_d, shopping_fw, shopping_atx])
grocery_tx = pd.concat([grocery_d, grocery_fw, grocery_atx])
resturant_tx = pd.concat([resturant_d, resturant_fw, resturant_atx])
golf_tx = pd.concat([golf_d, golf_fw, golf_atx])
school_tx = pd.concat([schools_dallas, schools_fortworth, schools_atx])


houses_tx['dist_hospitals'] = getDist(houses_tx, hospitals_tx)
houses_tx['dist_gym'] = getDist(houses_tx, gym_tx)
houses_tx['dist_cemetary'] = getDist(houses_tx, cemetary_tx)
houses_tx['dist_parks'] = getDist(houses_tx, parks_tx)
houses_tx['dist_beaches'] = getDist(houses_tx, beaches_tx)
houses_tx['dist_shopping'] = getDist(houses_tx, shopping_tx)
houses_tx['dist_grocery'] = getDist(houses_tx, grocery_tx)
houses_tx['dist_resturant'] = getDist(houses_tx, resturant_tx)
houses_tx['dist_golf'] = getDist(houses_tx, golf_tx)
houses_tx['dist_school'] = getDist(houses_tx, school_tx)

houses_tx = houses_tx.iloc[: , 1:]

In [141]:
#washington - dc
hospitals_dc = pd.concat([hospitals_dc, hospitals_ava, hospitals_alex])
gym_dc = pd.concat([gym_dc, gym_ava, gym_alex])
cemetary_dc = pd.concat([cemetary_dc, cemetary_ava, cemetary_alex])
parks_dc = pd.concat([parks_dc, parks_ava, parks_alex])
beaches_dc = pd.concat([beaches_dc, beaches_ava, beaches_alex])
shopping_dca = pd.concat([shopping_dc, shopping_ava, shopping_alex])
grocery_dc = pd.concat([grocery_dc, grocery_ava, grocery_alex])
resturant_dc = pd.concat([resturant_dc, resturant_ava, resturant_alex])
golf_dc = pd.concat([golf_dc, golf_ava, golf_alex])
school_dc = pd.concat([schools_washington, schools_ava, schools_alex])


houses_dc['dist_hospitals'] = getDist(houses_dc, hospitals_dc)
houses_dc['dist_gym'] = getDist(houses_dc, gym_dc)
houses_dc['dist_cemetary'] = getDist(houses_dc, cemetary_dc)
houses_dc['dist_parks'] = getDist(houses_dc, parks_dc)
houses_dc['dist_beaches'] = getDist(houses_dc, beaches_dc)
houses_dc['dist_shopping'] = getDist(houses_dc, shopping_dc)
houses_dc['dist_grocery'] = getDist(houses_dc, grocery_dc)
houses_dc['dist_resturant'] = getDist(houses_dc, resturant_dc)
houses_dc['dist_golf'] = getDist(houses_dc, golf_dc)
houses_dc['dist_school'] = getDist(houses_dc, school_dc)

houses_dc = houses_dc.iloc[: , 1:]

In [142]:
#Turned data to csv, put into final data

#houses_ca.to_csv("mi_final_lametro.csv")
#houses_il.to_csv("mi_final_chimetro.csv")
#houses_tx.to_csv("mi_final_txmetro.csv")
#houses_dc.to_csv("mi_final_dcmetro.csv")

In [74]:
#EDA

#transform
np.seterr(divide = 'ignore') 
houses_ca = pd.read_csv('../data/final/metro/mi_final_lametro.csv')
houses_ca['log'] = np.log(houses_ca['Norm Price'])
houses_ca['sqrt'] = np.sqrt(houses_ca['Norm Price'])
houses_ca['cube_root'] = np.power(houses_ca['Norm Price'], 1/3)

houses_il = pd.read_csv('../data/final/metro/mi_final_chimetro.csv')
houses_il['log'] = np.log(houses_il['Norm Price'])
houses_il['sqrt'] = np.sqrt(houses_il['Norm Price'])
houses_il['cube_root'] = np.power(houses_il['Norm Price'], 1/3)

houses_tx = pd.read_csv('../data/final/metro/mi_final_txmetro.csv')
houses_tx['log'] = np.log(houses_tx['Norm Price'])
houses_tx['sqrt'] = np.sqrt(houses_tx['Norm Price'])
houses_tx['cube_root'] = np.power(houses_tx['Norm Price'], 1/3)

houses_dc = pd.read_csv('../data/final/metro/mi_final_dcmetro.csv')
houses_dc['log'] = np.log(houses_dc['Norm Price'])
houses_dc['sqrt'] = np.sqrt(houses_dc['Norm Price'])
houses_dc['cube_root'] = np.power(houses_dc['Norm Price'], 1/3)

houses_ca = houses_ca[['Address','City', 'Longitude','Latitude', 'SQ Ft', 'Housing Price','Housing Price/SQ Ft', 'Norm Price', 'log', 'sqrt', 'cube_root', 'dist_hospitals', 'dist_gym', 'dist_cemetary', 'dist_parks', 'dist_beaches', 'dist_shopping', 'dist_grocery', 'dist_resturant', 'dist_golf', 'dist_school']]
houses_il = houses_il[['Address','City', 'Longitude','Latitude', 'SQ Ft', 'Housing Price','Housing Price/SQ Ft', 'Norm Price', 'log', 'sqrt', 'cube_root', 'dist_hospitals', 'dist_gym', 'dist_cemetary', 'dist_parks', 'dist_beaches', 'dist_shopping', 'dist_grocery', 'dist_resturant', 'dist_golf', 'dist_school']]
houses_tx = houses_tx[['Address','City', 'Longitude','Latitude', 'SQ Ft', 'Housing Price','Housing Price/SQ Ft', 'Norm Price', 'log', 'sqrt', 'cube_root', 'dist_hospitals', 'dist_gym', 'dist_cemetary', 'dist_parks', 'dist_beaches', 'dist_shopping', 'dist_grocery', 'dist_resturant', 'dist_golf', 'dist_school']]
houses_dc = houses_dc[['Address','City', 'Longitude','Latitude', 'SQ Ft', 'Housing Price','Housing Price/SQ Ft', 'Norm Price', 'log', 'sqrt', 'cube_root', 'dist_hospitals', 'dist_gym', 'dist_cemetary', 'dist_parks', 'dist_beaches', 'dist_shopping', 'dist_grocery', 'dist_resturant', 'dist_golf', 'dist_school']]

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
sns.heatmap(houses_ca.corr(), annot=True)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
sns.heatmap(houses_il.corr(), annot=True)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
sns.heatmap(houses_tx.corr(), annot=True)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
sns.heatmap(houses_dc.corr(), annot=True)

In [83]:
#separate to explore
houses_ca_eda = houses_ca.iloc[:,19:]
houses_il_eda = houses_il.iloc[:,19:]
houses_tx_eda = houses_tx.iloc[:,19:]
houses_dc_eda = houses_dc.iloc[:,19:]

houses_ca_eda = pd.concat([houses_ca['Norm Price'], houses_ca_eda], axis = 1)
houses_ca_eda = pd.concat([houses_ca['Housing Price'], houses_ca_eda], axis = 1)

houses_il_eda = pd.concat([houses_il['Norm Price'], houses_il_eda], axis = 1)
houses_il_eda = pd.concat([houses_il['Housing Price'], houses_il_eda], axis = 1)

houses_tx_eda = pd.concat([houses_tx['Norm Price'], houses_tx_eda], axis = 1)
houses_tx_eda = pd.concat([houses_tx['Housing Price'], houses_tx_eda], axis = 1)

houses_dc_eda = pd.concat([houses_dc['Norm Price'], houses_dc_eda], axis = 1)
houses_dc_eda = pd.concat([houses_dc['Housing Price'], houses_dc_eda], axis = 1)

In [ ]:
sns.pairplot(houses_ca_eda)

In [ ]:
sns.pairplot(houses_il_eda)

In [ ]:
sns.pairplot(houses_tx_eda)

In [ ]:
sns.pairplot(houses_dc_eda)

### **Regression Analysis**


**Data Manipulation** 

Combining all metro datas into one

In [ ]:
## Combining all METRO Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
 
lametro = pd.read_csv("../data/final/metro/mi_final_lametro.csv")
chimetro = pd.read_csv("../data/final/metro/mi_final_chimetro.csv")
dcmetro = pd.read_csv("../data/final/metro/mi_final_dcmetro.csv")
dalmetro = pd.read_csv("../data/final/metro/mi_final_txmetro.csv")

lametro_adj = lametro[["Norm Price","City","dist_hospitals","dist_gym","dist_cemetary","dist_parks","dist_beaches","dist_shopping","dist_grocery","dist_resturant","dist_golf","dist_school"]]
chimetro_adj = chimetro[["Norm Price","City","dist_hospitals","dist_gym","dist_cemetary","dist_parks","dist_beaches","dist_shopping","dist_grocery","dist_resturant","dist_golf","dist_school"]]
dcmetro_adj = dcmetro[["Norm Price","City","dist_hospitals","dist_gym","dist_cemetary","dist_parks","dist_beaches","dist_shopping","dist_grocery","dist_resturant","dist_golf","dist_school"]]
dalmetro_adj = dalmetro[["Norm Price","City","dist_hospitals","dist_gym","dist_cemetary","dist_parks","dist_beaches","dist_shopping","dist_grocery","dist_resturant","dist_golf","dist_school"]]

lametro_adj = lametro_adj.rename(columns={lametro_adj.columns[0]: "Norm" })
chimetro_adj = chimetro_adj.rename(columns={chimetro_adj.columns[0]: "Norm" })
dcmetro_adj = dcmetro_adj.rename(columns={dcmetro_adj.columns[0]: "Norm" })
dalmetro_adj = dalmetro_adj.rename(columns={dalmetro_adj.columns[0]: "Norm" })

metro = pd.concat([lametro_adj,chimetro_adj,dcmetro_adj,dalmetro_adj])
#metro.to_csv("metro.csv")

nummetro = metro.drop(columns = ["City","Norm"])
nummetro = nummetro.reset_index(drop=True)
citymetro = metro[["City","Norm"]]
citymetro = citymetro.reset_index(drop=True)
nummetro = nummetro[(np.abs(stats.zscore(nummetro)) < 3).all(axis=1)]
nummetro = nummetro.join(citymetro)
nummetro = nummetro.drop(nummetro.index[nummetro['Norm'] <= 0])

**Model I**

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from scipy import stats

resulta = sm.OLS(nummetro['Norm'], nummetro.drop(columns=["Norm","City"])).fit() #Initialize the GLS 
print(resulta.summary())

Assumption Check:

In [ ]:
# Multicollinearity: VIF
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
y, X = dmatrices('Norm ~ dist_hospitals+dist_gym+dist_cemetary+dist_parks+dist_beaches+dist_shopping+dist_grocery+dist_resturant+dist_golf+dist_school', data=nummetro, return_type='dataframe')
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns
vif = vif.set_index("variable")
vif
vif.style.applymap(lambda x: 'background-color : purple' if x>5 else '')

In [ ]:
# Residual Plots
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def linearity_test(model, y):
    '''
    Function for visually inspecting the assumption of linearity in a linear regression model.
    It plots observed vs. predicted values and residuals vs. predicted values.
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Observed vs. Predicted Values', fontsize=16)
    ax[0].set(xlabel='Predicted', ylabel='Observed')

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Residuals vs. Predicted Values', fontsize=16)
    ax[1].set(xlabel='Predicted', ylabel='Residuals')
    
linearity_test(resulta, y=nummetro[["Norm"]])    

In [ ]:
# Unequal Variances Check: BP
import statsmodels.stats.api as sms
sms.het_breuschpagan(resulta.resid, resulta.model.exog)

In [ ]:
# Normality: QQ Plot
import statsmodels.stats.api
stats.probplot(resulta.resid, dist="norm", plot= plt)
plt.title("Model1 Residuals Q-Q Plot")

**Model II: Reciprocal Transformation**

In [ ]:
# DATA MANIPULATION
import numpy as np
nummetro = metro.drop(columns = ["City","Norm"])
nummetro = np.exp(-nummetro)
Q1 = nummetro.quantile(q=.25)
Q3 = nummetro.quantile(q=.75)
IQR = nummetro.apply(stats.iqr)
nummetro = nummetro[~((nummetro < (Q1-1.5*IQR)) | (nummetro > (Q3+1.5*IQR))).any(axis=1)]
nummetro = nummetro.reset_index(drop=True)
citymetro = metro[["City","Norm"]]
citymetro["Norm"] = np.exp(-citymetro["Norm"])
citymetro = citymetro.reset_index(drop=True)

nummetro = nummetro.join(citymetro)
#nummetro.to_csv("test.csv")

In [ ]:
# Regression Analysis
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from scipy import stats

nummetro = nummetro[nummetro.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)] 
resulta = sm.OLS(nummetro['Norm'], nummetro.drop(columns=["Norm","City"])).fit() 
print(resulta.summary())

In [ ]:
# Unequal Variances: BP
import statsmodels.stats.api as sms
sms.het_breuschpagan(resulta.resid, resulta.model.exog)


In [ ]:
# Multicollinearity: VIF
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
y, X = dmatrices('Norm ~ dist_hospitals+dist_gym+dist_cemetary+dist_parks+dist_beaches+dist_shopping+dist_grocery+dist_resturant+dist_golf+dist_school', data=nummetro, return_type='dataframe')
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns
vif = vif.set_index("variable")
vif
vif.style.applymap(lambda x: 'background-color : purple' if x>5 else '')

In [ ]:
# Residual Plots
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

linearity_test(resulta, y=nummetro[["Norm"]])    

In [ ]:
# Normality: QQ Plot
import statsmodels.stats.api
stats.probplot(resulta.resid, dist="norm", plot= plt)
plt.title("Model1 Residuals Q-Q Plot")

**Initial Models: Separate Metro Areas**

In [ ]:
# Los Angeles Metro Area
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from scipy import stats

citymetro = lametro_adj[["Norm","City"]]
citymetro = citymetro.reset_index(drop=True)
lametro_adj = lametro_adj.drop(columns = ["City","Norm"])
lametro_adj = lametro_adj.reset_index(drop=True)
lametro_adj = lametro_adj[(np.abs(stats.zscore(lametro_adj)) < 3).all(axis=1)]
lametro_adj = lametro_adj.join(citymetro)
resultla = sm.OLS(lametro_adj['Norm'], lametro_adj.drop(columns=["Norm","City"])).fit() #Initialize the GLS 
print(resultla.summary())

In [ ]:
# DC Metro Area
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from scipy import stats

citymetro = dcmetro_adj[["Norm","City"]]
citymetro = citymetro.reset_index(drop=True)
dcmetro_adj = dcmetro_adj.drop(columns = ["City","Norm"])
dcmetro_adj = dcmetro_adj.reset_index(drop=True)
dcmetro_adj = dcmetro_adj[(np.abs(stats.zscore(dcmetro_adj)) < 3).all(axis=1)]
dcmetro_adj = dcmetro_adj.join(citymetro)
resultdc = sm.OLS(dcmetro_adj['Norm'], dcmetro_adj.drop(columns=["Norm","City"])).fit() #Initialize the GLS 
print(resultdc.summary())

In [ ]:
# Dallas Metro Area
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from scipy import stats

citymetro = dalmetro_adj[["Norm","City"]]
citymetro = citymetro.reset_index(drop=True)
dalmetro_adj = dalmetro_adj.drop(columns = ["City","Norm"])
dalmetro_adj = dalmetro_adj.reset_index(drop=True)
dalmetro_adj = dalmetro_adj[(np.abs(stats.zscore(dalmetro_adj)) < 3).all(axis=1)]
dalmetro_adj = dalmetro_adj.join(citymetro)
resultdal = sm.OLS(dalmetro_adj['Norm'], dalmetro_adj.drop(columns=["Norm","City"])).fit() #Initialize the GLS 
print(resultdal.summary())

In [ ]:
# Chicago Metro Area
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from scipy import stats

citymetro = chimetro_adj[["Norm","City"]]
citymetro = citymetro.reset_index(drop=True)
chimetro_adj = chimetro_adj.drop(columns = ["City","Norm"])
chimetro_adj = chimetro_adj.reset_index(drop=True)
chimetro_adj = chimetro_adj[(np.abs(stats.zscore(chimetro_adj)) < 3).all(axis=1)]
chimetro_adj = chimetro_adj.join(citymetro)
resultchi = sm.OLS(chimetro_adj['Norm'], chimetro_adj.drop(columns=["Norm","City"])).fit() #Initialize the GLS 
print(resultchi.summary())

In [ ]:
# VIF Check for All Four Metro Areas
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
yla, Xla = dmatrices('Norm ~ dist_hospitals+dist_gym+dist_cemetary+dist_parks+dist_beaches+dist_shopping+dist_grocery+dist_resturant+dist_golf+dist_school', data=lametro_adj, return_type='dataframe')
ydc, Xdc = dmatrices('Norm ~ dist_hospitals+dist_gym+dist_cemetary+dist_parks+dist_beaches+dist_shopping+dist_grocery+dist_resturant+dist_golf+dist_school', data=dcmetro_adj, return_type='dataframe')
ydal, Xdal = dmatrices('Norm ~ dist_hospitals+dist_gym+dist_cemetary+dist_parks+dist_beaches+dist_shopping+dist_grocery+dist_resturant+dist_golf+dist_school', data=dalmetro_adj, return_type='dataframe')
ychi, Xchi = dmatrices('Norm ~ dist_hospitals+dist_gym+dist_cemetary+dist_parks+dist_beaches+dist_shopping+dist_grocery+dist_resturant+dist_golf+dist_school', data=chimetro_adj, return_type='dataframe')
vif = pd.DataFrame()
vif['VIF LA'] = [variance_inflation_factor(Xla.values, i) for i in range(Xla.shape[1])]
vif['VIF DC'] = [variance_inflation_factor(Xdc.values, i) for i in range(Xdc.shape[1])]
vif['VIF Dal'] = [variance_inflation_factor(Xdal.values, i) for i in range(Xdal.shape[1])]
vif['VIF CHI'] = [variance_inflation_factor(Xchi.values, i) for i in range(Xchi.shape[1])]
vif['variable'] = Xla.columns
vif = vif.set_index("variable")
vif
vif.style.applymap(lambda x: 'background-color : purple' if x>5 else '')

In [ ]:
# UNEQUAL VARIANCE for all Metro Areas
import statsmodels.stats.api as sms
la = sms.het_breuschpagan(resultla.resid, resultla.model.exog)
dc = sms.het_breuschpagan(resultdc.resid, resultdc.model.exog)
dal = sms.het_breuschpagan(resultdal.resid, resultdal.model.exog)
chi = sms.het_breuschpagan(resultchi.resid, resultchi.model.exog)
print(la,dc,dal,chi)

In [ ]:
# NORMALITY for all Metro Areas
import statsmodels.stats.api
la = statsmodels.stats.stattools.jarque_bera(resultla.resid)
dc = statsmodels.stats.stattools.jarque_bera(resultdc.resid)
dal = statsmodels.stats.stattools.jarque_bera(resultdal.resid)
chi = statsmodels.stats.stattools.jarque_bera(resultchi.resid)
print(la,dc,dal,chi)

**Model II for separate metros**

In [ ]:
# Los Angeles
import numpy as np
nummetro = lametro_adj.drop(columns = ["City","Norm"])
nummetro = np.exp(-nummetro)
Q1 = nummetro.quantile(q=.25)
Q3 = nummetro.quantile(q=.75)
IQR = nummetro.apply(stats.iqr)
nummetro = nummetro[~((nummetro < (Q1-1.5*IQR)) | (nummetro > (Q3+1.5*IQR))).any(axis=1)]
nummetro = nummetro.reset_index(drop=True)
citymetro = lametro_adj[["City","Norm"]]
citymetro["Norm"] = np.exp(-citymetro["Norm"])
citymetro = citymetro.reset_index(drop=True)
nummetro = nummetro.join(citymetro)
resultla = sm.OLS(nummetro['Norm'], nummetro.drop(columns=["Norm","City"])).fit() #Initialize the GLS 
print(resultla.summary())

In [ ]:
# DC
import numpy as np
nummetro = dcmetro_adj.drop(columns = ["City","Norm"])
nummetro = np.exp(-nummetro)
Q1 = nummetro.quantile(q=.25)
Q3 = nummetro.quantile(q=.75)
IQR = nummetro.apply(stats.iqr)
nummetro = nummetro[~((nummetro < (Q1-1.5*IQR)) | (nummetro > (Q3+1.5*IQR))).any(axis=1)]
nummetro = nummetro.reset_index(drop=True)
citymetro = dcmetro_adj[["City","Norm"]]
citymetro["Norm"] = np.exp(-citymetro["Norm"])
citymetro = citymetro.reset_index(drop=True)
nummetro = nummetro.join(citymetro)
resultdc = sm.OLS(nummetro['Norm'], nummetro.drop(columns=["Norm","City","dist_hospitals","dist_school"])).fit() #Initialize the GLS 
print(resultdc.summary())

In [ ]:
# Chicago
import numpy as np
nummetro = chimetro_adj.drop(columns = ["City","Norm"])
nummetro = np.exp(-nummetro)
Q1 = nummetro.quantile(q=.25)
Q3 = nummetro.quantile(q=.75)
IQR = nummetro.apply(stats.iqr)
nummetro = nummetro[~((nummetro < (Q1-1.5*IQR)) | (nummetro > (Q3+1.5*IQR))).any(axis=1)]
nummetro = nummetro.reset_index(drop=True)
citymetro = chimetro_adj[["City","Norm"]]
citymetro["Norm"] = np.exp(-citymetro["Norm"])
citymetro = citymetro.reset_index(drop=True)
nummetro = nummetro.join(citymetro)
resultchi = sm.OLS(nummetro['Norm'], nummetro.drop(columns=["Norm","City","dist_cemetary","dist_beaches","dist_shopping","dist_grocery"])).fit() #Initialize the GLS 
print(resultchi.summary())

In [ ]:
# Dallas
import numpy as np
nummetro = dalmetro_adj.drop(columns = ["City","Norm"])
#nummetro = np.exp(-nummetro)
Q1 = nummetro.quantile(q=.25)
Q3 = nummetro.quantile(q=.75)
IQR = nummetro.apply(stats.iqr)
nummetro = nummetro[~((nummetro < (Q1-1.5*IQR)) | (nummetro > (Q3+1.5*IQR))).any(axis=1)]
nummetro = nummetro.reset_index(drop=True)
citymetro = dalmetro_adj[["City","Norm"]]
citymetro["Norm"] = np.exp(-citymetro["Norm"])
citymetro = citymetro.reset_index(drop=True)
nummetro = nummetro.join(citymetro)
resultdal = sm.OLS(nummetro['Norm'], nummetro.drop(columns=["Norm","City"])).fit() #Initialize the GLS 
print(resultdal.summary())

In [ ]:
# Normality Check for LA & Dallas (not shown on summary)
import statsmodels.stats.api
la = statsmodels.stats.stattools.jarque_bera(resultla.resid)
dc = statsmodels.stats.stattools.jarque_bera(resultdc.resid)
chi = statsmodels.stats.stattools.jarque_bera(resultchi.resid)
dal = statsmodels.stats.stattools.jarque_bera(resultdal.resid)
print(la,dc,chi,dal)

In [ ]:
# UNEQUAL VARIANCE for all Metro Areas
import statsmodels.stats.api as sms
la = sms.het_breuschpagan(resultla.resid, resultla.model.exog)
dc = sms.het_breuschpagan(resultdc.resid, resultdc.model.exog)
dal = sms.het_breuschpagan(resultdal.resid, resultdal.model.exog)
chi = sms.het_breuschpagan(resultchi.resid, resultchi.model.exog)
print(la,dc,dal,chi)

### Additional Codes (Interactive Visualization)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

lametro = pd.read_csv("../data/final/final_lametro.csv")
dcmetro = pd.read_csv("../data/final/final_dcmetro.csv")
chimetro = pd.read_csv("../data/final/final_chimetro.csv")
dalmetro = pd.read_csv("../data/final/final_txmetro.csv")


lametro_adj = lametro[["Norm Price", "City", "Longitude", "Latitude"]]
lametro_adj["Type"] = "House"
dcmetro_adj = dcmetro[["Norm Price", "City", "Longitude", "Latitude"]]
dcmetro_adj["Type"] = "House"
chimetro_adj = chimetro[["Norm Price", "City", "Longitude", "Latitude"]]
chimetro_adj["Type"] = "House"
dalmetro_adj = dalmetro[["Norm Price", "City", "Longitude", "Latitude"]]
dalmetro_adj["Type"] = "House"


lametro_adj = lametro_adj.rename(columns={lametro_adj.columns[0]: "Norm"})
dcmetro_adj = dcmetro_adj.rename(columns={dcmetro_adj.columns[0]: "Norm"})
chimetro_adj = chimetro_adj.rename(columns={dcmetro_adj.columns[0]: "Norm"})
dalmetro_adj = dalmetro_adj.rename(columns={dcmetro_adj.columns[0]: "Norm"})



In [ ]:
houses = pd.read_csv('../data/final/houses_clean.csv')

houses_la = houses[houses['City'] == 'Los Angeles']
houses_anaheim = houses[houses['City'] == 'Anaheim']
houses_longbeach = houses[houses['City'] == 'Long Beach']

houses_dallas = houses[houses['City'] == 'Dallas']
houses_fortworth = houses[houses['City'] == 'Fort Worth']
houses_arlington_tx = houses[houses['City'] == 'ArlingtonTX']

houses_chicago = houses[houses['City'] == 'Chicago']
houses_naperville = houses[houses['City'] == 'Naperville']
houses_elgin = houses[houses['City'] == 'Elgin']

houses_washington = houses[houses['City'] == 'Washington']
houses_alexandria = houses[houses['City'] == 'Alexandria']
houses_arlington_va = houses[houses['City'] == 'ArlingtonVA']


schools = pd.read_csv('../data/final/schools.csv')
schools_la = schools[schools['City'] == 'Los Angeles']
schools_anaheim = schools[schools['City'] == 'Anaheim']
schools_longbeach = schools[schools['City'] == 'Long Beach']

schools_dallas = schools[schools['City'] == 'Dallas']
schools_fortworth = schools[schools['City'] == 'Fort Worth']
schools_atx = schools[schools['City'] == 'ArlingtonTX']

schools_chicago = schools[schools['City'] == 'Chicago']
schools_naperville = schools[schools['City'] == 'Naperville']
schools_elgin = schools[schools['City'] == 'Elgin']

schools_washington = schools[schools['City'] == 'Washington']
schools_alex = schools[schools['City'] == 'Alexandria']
schools_ava = schools[schools['City'] == 'ArlingtonVA']


hospitals = pd.read_csv('../data/final/hospital.csv')

hospitals_la = hospitals[hospitals['City'] == 'Los Angeles']
hospitals_a = hospitals[hospitals['City'] == 'Anaheim']
hospitals_lb = hospitals[hospitals['City'] == 'Long Beach']

hospitals_d = hospitals[hospitals['City'] == 'Dallas']
hospitals_fw = hospitals[hospitals['City'] == 'Fort Worth']
hospitals_atx = hospitals[hospitals['City'] == 'ArlingtonTX']

hospitals_c = hospitals[hospitals['City'] == 'Chicago']
hospitals_n = hospitals[hospitals['City'] == 'Naperville']
hospitals_e = hospitals[hospitals['City'] == 'Elgin']

hospitals_dc = schools[schools['City'] == 'Washington']
hospitals_alex = schools[schools['City'] == 'Alexandria']
hospitals_ava = schools[schools['City'] == 'ArlingtonVA']

grocery = pd.read_csv('../data/final/grocery.csv')

grocery_la = grocery[grocery['City'] == 'Los Angeles']
grocery_a = grocery[grocery['City'] == 'Anaheim']
grocery_lb = grocery[grocery['City'] == 'Long Beach']

grocery_d = grocery[grocery['City'] == 'Dallas']
grocery_fw = grocery[grocery['City'] == 'Fort Worth']
grocery_atx = grocery[grocery['City'] == 'ArlingtonTX']

grocery_c = grocery[grocery['City'] == 'Chicago']
grocery_n = grocery[grocery['City'] == 'Naperville']
grocery_e = grocery[grocery['City'] == 'Elgin']

grocery_dc = grocery[grocery['City'] == 'Washington']
grocery_alex = grocery[grocery['City'] == 'Alexandria']
grocery_ava = grocery[grocery['City'] == 'ArlingtonVA']

gym = pd.read_csv('../data/final/gym.csv')

gym_la = gym[gym['City'] == 'Los Angeles']
gym_lb = gym[gym['City'] == 'Long Beach']
gym_a = gym[gym['City'] == 'Anaheim']

gym_d = gym[gym['City'] == 'Dallas']
gym_atx = gym[gym['City'] == 'ArlingtonTX']
gym_fw = gym[gym['City'] == 'Fort Worth']

gym_c = gym[gym['City'] == 'Chicago']
gym_e = gym[gym['City'] == 'Elgin']
gym_n = gym[gym['City'] == 'Naperville']

gym_dc = gym[gym['City'] == 'Washington']
gym_ava = gym[gym['City'] == 'ArlingtonVA']
gym_alex = gym[gym['City'] == 'Alexandria']

parks = pd.read_csv('../data/final/parks.csv')

parks_la = parks[parks['City'] == 'Los Angeles']
parks_lb = parks[parks['City'] == 'Long Beach']
parks_a = parks[parks['City'] == 'Anaheim']

parks_d = parks[parks['City'] == 'Dallas']
parks_atx = parks[parks['City'] == 'ArlingtonTX']
parks_fw = parks[parks['City'] == 'Fort Worth']

parks_c = parks[parks['City'] == 'Chicago']
parks_e = parks[parks['City'] == 'Elgin']
parks_n = parks[parks['City'] == 'Naperville']

parks_dc = parks[parks['City'] == 'Washington']
parks_ava = parks[parks['City'] == 'ArlingtonVA']
parks_alex = parks[parks['City'] == 'Alexandria']

beaches = pd.read_csv('../data/final/beach.csv')

beaches_la = beaches[beaches['City'] == 'Los Angeles']
beaches_lb = beaches[beaches['City'] == 'Long Beach']
beaches_a = beaches[beaches['City'] == 'Anaheim']

beaches_d = beaches[beaches['City'] == 'Dallas']
beaches_atx = beaches[beaches['City'] == 'ArlingtonTX']
beaches_fw = beaches[beaches['City'] == 'Fort Worth']

beaches_c = beaches[beaches['City'] == 'Chicago']
beaches_e = beaches[beaches['City'] == 'Elgin']
beaches_n = beaches[beaches['City'] == 'Naperville']

beaches_dc = beaches[beaches['City'] == 'Washington']
beaches_ava = beaches[beaches['City'] == 'ArlingtonVA']
beaches_alex = beaches[beaches['City'] == 'Alexandria']

cemetary = pd.read_csv('../data/final/cemetary.csv')

cemetary_la = cemetary[cemetary['City'] == 'Los Angeles']
cemetary_lb = cemetary[cemetary['City'] == 'Long Beach']
cemetary_a = cemetary[cemetary['City'] == 'Anaheim']

cemetary_d = cemetary[cemetary['City'] == 'Dallas']
cemetary_atx = cemetary[cemetary['City'] == 'ArlingtonTX']
cemetary_fw = cemetary[cemetary['City'] == 'Fort Worth']

cemetary_c = cemetary[cemetary['City'] == 'Chicago']
cemetary_e = cemetary[cemetary['City'] == 'Elgin']
cemetary_n = cemetary[cemetary['City'] == 'Naperville']

cemetary_dc = cemetary[cemetary['City'] == 'Washington']
cemetary_ava = cemetary[cemetary['City'] == 'ArlingtonVA']
cemetary_alex = cemetary[cemetary['City'] == 'Alexandria']

shopping = pd.read_csv('../data/final/shopping.csv')

shopping_la = shopping[shopping['City'] == 'Los Angeles']
shopping_lb = shopping[shopping['City'] == 'Long Beach']
shopping_a = shopping[shopping['City'] == 'Anaheim']

shopping_d = shopping[shopping['City'] == 'Dallas']
shopping_atx = shopping[shopping['City'] == 'ArlingtonTX']
shopping_fw = shopping[shopping['City'] == 'Fort Worth']

shopping_c = shopping[shopping['City'] == 'Chicago']
shopping_e = shopping[shopping['City'] == 'Elgin']
shopping_n = shopping[shopping['City'] == 'Naperville']

shopping_dc = shopping[shopping['City'] == 'Washington']
shopping_ava = shopping[shopping['City'] == 'ArlingtonVA']
shopping_alex = shopping[shopping['City'] == 'Alexandria']

resturant = pd.read_csv('../data/final/restaurants.csv')

resturant_la = resturant[resturant['City'] == 'Los Angeles']
resturant_lb = resturant[resturant['City'] == 'Long Beach']
resturant_a = resturant[resturant['City'] == 'Anaheim']

resturant_d = resturant[resturant['City'] == 'Dallas']
resturant_atx = resturant[resturant['City'] == 'ArlingtonTX']
resturant_fw = resturant[resturant['City'] == 'Fort Worth']

resturant_c = resturant[resturant['City'] == 'Chicago']
resturant_e = resturant[resturant['City'] == 'Elgin']
resturant_n = resturant[resturant['City'] == 'Naperville']

resturant_dc = resturant[resturant['City'] == 'Washington']
resturant_ava = resturant[resturant['City'] == 'ArlingtonVA']
resturant_alex = resturant[resturant['City'] == 'Alexandria']

golf = pd.read_csv('../data/final/golf.csv')

golf_la = golf[golf['City'] == 'Los Angeles']
golf_lb = golf[golf['City'] == 'Long Beach']
golf_a = golf[golf['City'] == 'Anaheim']

golf_d = golf[golf['City'] == 'Dallas']
golf_atx = golf[golf['City'] == 'ArlingtonTX']
golf_fw = golf[golf['City'] == 'Fort Worth']

golf_c = golf[golf['City'] == 'Chicago']
golf_e = golf[golf['City'] == 'Elgin']
golf_n = golf[golf['City'] == 'Naperville']

golf_dc = golf[golf['City'] == 'Washington']
golf_ava = golf[golf['City'] == 'ArlingtonVA']
golf_alex = golf[golf['City'] == 'Alexandria']

hospitals_ca = pd.concat([hospitals_la, hospitals_lb, hospitals_a])
hospitals_ca["Type"] = "Hospitals"
hospitals_ca = hospitals_ca.dropna()
gym_ca = pd.concat([gym_la, gym_lb, gym_a])
gym_ca["Type"] = "Gym"
gym_ca = gym_ca.dropna()
cemetary_ca = pd.concat([cemetary_la, cemetary_lb, cemetary_a])
cemetary_ca["Type"] = "Cemetary"
cemetary_ca = cemetary_ca.dropna()

parks_ca = pd.concat([parks_la, parks_lb, parks_la])
parks_ca["Type"] = "Parks"
parks_ca = parks_ca.dropna()

beaches_ca = pd.concat([beaches_la, beaches_lb, beaches_a])
beaches_ca["Type"] = "Beaches"
beaches_ca = beaches_ca.dropna()

shopping_ca = pd.concat([shopping_la, shopping_lb, shopping_a])
shopping_ca["Type"] = "Shopping"
shopping_ca = shopping_ca.dropna()

grocery_ca = pd.concat([grocery_la, grocery_lb, grocery_a])
grocery_ca["Type"] = "Grocery"
grocery_ca = grocery_ca.dropna()

resturant_ca = pd.concat([resturant_la, resturant_lb, resturant_a])
resturant_ca["Type"] = "Restaurants"
resturant_ca = resturant_ca.dropna()

golf_ca = pd.concat([golf_la, golf_lb, golf_a])
golf_ca["Type"] = "Golf"
golf_ca = golf_ca.dropna()

school_ca = pd.concat([schools_la, schools_longbeach, schools_anaheim])
school_ca["Type"] = "School"
school_ca = school_ca.dropna()


hospitals_dc = pd.concat([hospitals_dc, hospitals_ava, hospitals_alex])
hospitals_dc["Type"] = "Hospitals"
hospitals_dc = hospitals_dc.dropna()
gym_dc = pd.concat([gym_dc, gym_ava, gym_alex])
gym_dc["Type"] = "Gym"
gym_dc = gym_dc.dropna()
cemetary_dc = pd.concat([cemetary_dc, cemetary_ava, cemetary_alex])
cemetary_dc["Type"] = "Cemetary"
cemetary_dc = cemetary_dc.dropna()
parks_dc = pd.concat([parks_dc, parks_ava, parks_alex])
parks_dc["Type"] = "Parks"
parks_dc = parks_dc.dropna()
beaches_dc = pd.concat([beaches_dc, beaches_ava, beaches_alex])
beaches_dc["Type"] = "Beaches"
beaches_dc = beaches_dc.dropna()
shopping_dc= pd.concat([shopping_dc, shopping_ava, shopping_alex])
shopping_dc["Type"] = "Shopping"
shopping_dc = shopping_dc.dropna()
grocery_dc = pd.concat([grocery_dc, grocery_ava, grocery_alex])
grocery_dc["Type"] = "Grocery"
grocery_dc = grocery_dc.dropna()
resturant_dc = pd.concat([resturant_dc, resturant_ava, resturant_alex])
resturant_dc["Type"] = "Restaurants"
resturant_dc = resturant_dc.dropna()
golf_dc = pd.concat([golf_dc, golf_ava, golf_alex])
golf_dc["Type"] = "Golf"
golf_dc = golf_dc.dropna()
school_dc = pd.concat([schools_washington, schools_ava, schools_alex])
school_dc["Type"] = "School"
school_dc = school_dc.dropna()

hospitals_chi = pd.concat([hospitals_c, hospitals_e, hospitals_n])
hospitals_chi["Type"] = "Hospitals"
hospitals_chi = hospitals_chi.dropna()
gym_chi = pd.concat([gym_c, gym_e, gym_n])
gym_chi["Type"] = "Gym"
gym_chi = gym_chi.dropna()
cemetary_chi = pd.concat([cemetary_c, cemetary_e, cemetary_n])
cemetary_chi["Type"] = "Cemetary"
cemetary_chi = cemetary_chi.dropna()
parks_chi = pd.concat([parks_c, parks_e, parks_n])
parks_chi["Type"] = "Parks"
parks_chi = parks_chi.dropna()
beaches_chi = pd.concat([beaches_c, beaches_e, beaches_n])
beaches_chi["Type"] = "Beaches"
beaches_chi = beaches_chi.dropna()
shopping_chi= pd.concat([shopping_c, shopping_e, shopping_n])
shopping_chi["Type"] = "Shopping"
shopping_chi = shopping_chi.dropna()
grocery_chi = pd.concat([grocery_c, grocery_e, grocery_n])
grocery_chi["Type"] = "Grocery"
grocery_chi = grocery_dc.dropna()
resturant_chi = pd.concat([resturant_c, resturant_e, resturant_n])
resturant_chi["Type"] = "Restaurants"
resturant_chi = resturant_chi.dropna()
golf_chi = pd.concat([golf_c, golf_e, golf_n])
golf_chi["Type"] = "Golf"
golf_chi = golf_chi.dropna()
school_chi= pd.concat([schools_chicago, schools_naperville, schools_elgin])
school_chi["Type"] = "School"
school_chi = school_chi.dropna()

hospitals_dal = pd.concat([hospitals_d, hospitals_atx, hospitals_fw])
hospitals_dal["Type"] = "Hospitals"
hospitals_dal = hospitals_dal.dropna()
gym_dal = pd.concat([gym_d, gym_atx, gym_fw])
gym_dal["Type"] = "Gym"
gym_dal = gym_dal.dropna()
cemetary_dal = pd.concat([cemetary_d, cemetary_atx, cemetary_fw])
cemetary_dal["Type"] = "Cemetary"
cemetary_dal = cemetary_dal.dropna()
parks_dal = pd.concat([parks_d, parks_atx, parks_fw])
parks_dal["Type"] = "Parks"
parks_dal = parks_dal.dropna()
beaches_dal = pd.concat([beaches_d, beaches_atx, beaches_fw])
beaches_dal["Type"] = "Beaches"
beaches_dal = beaches_dal.dropna()
shopping_dal= pd.concat([shopping_d, shopping_atx, shopping_fw])
shopping_dal["Type"] = "Shopping"
shopping_dal = shopping_dal.dropna()
grocery_dal = pd.concat([grocery_d, grocery_atx, grocery_fw])
grocery_dal["Type"] = "Grocery"
grocery_dal = grocery_dal.dropna()
resturant_dal = pd.concat([resturant_d, resturant_atx, resturant_fw])
resturant_dal["Type"] = "Restaurants"
resturant_dal = resturant_chi.dropna()
golf_dal = pd.concat([golf_d, golf_atx, golf_fw])
golf_dal["Type"] = "Golf"
golf_dal = golf_dal.dropna()
school_dal= pd.concat([schools_dallas, schools_atx, schools_fortworth])
school_dal["Type"] = "School"
school_dal = school_dal.dropna()


### **MAP**: A visualization of the neighborhood and its houses.

Through the python package *folium*, here is an interactive maps of the locations of the housing and neighborhood landmarks of the Metro Areas.

**LOS ANGELES METRO AREA**


In [ ]:
import folium
m = folium.Map(location = [34.052235, -118.243683], zoom_start = 11)

cols = ["Type", "Longitude", "Latitude"]
for name, lat, lng in lametro_adj[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "red", radius = 10, popup = popup)
    m.add_child(circle)
for name, lng, lat in hospitals_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    blue = folium.Circle([float(lat), float(lng)], color = "blue", radius = 5, popup = popup)
    m.add_child(blue)
for name, lng, lat in gym_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "yellow", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in cemetary_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "green", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in parks_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "purple", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in beaches_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "black", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in shopping_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "orange", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in grocery_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "lightblue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in resturant_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "gray", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in golf_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "cadetblue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in school_ca[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "beige", radius = 5, popup = popup)
    m.add_child(circle)
    
fig = folium.Figure(width = 900, height = 400)
fig.add_child(m)

**DC METRO AREA**




In [ ]:
import folium


m = folium.Map(location = [38.900497,  -77.007507], zoom_start = 11)

cols = ["Type", "Longitude", "Latitude"]
for name, lat, lng in dcmetro_adj[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "red", radius = 10, popup = popup)
    m.add_child(circle)
for name, lng, lat in hospitals_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "blue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in gym_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "yellow", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in cemetary_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "green", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in parks_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "purple", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in beaches_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "black", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in shopping_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "orange", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in grocery_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "lightblue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in resturant_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "gray", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in golf_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "cadetblue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in school_dc[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "beige", radius = 5, popup = popup)
    m.add_child(circle)
    
fig = folium.Figure(width = 900, height = 400)
fig.add_child(m)
m

**CHICAGO METRO AREA**




In [ ]:
import folium


m = folium.Map(location = [41.8781,  -87.6298], zoom_start = 11)

cols = ["Type", "Longitude", "Latitude"]
for name, lat, lng in chimetro_adj[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "red", radius = 10, popup = popup)
    m.add_child(circle)
for name, lng, lat in hospitals_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "blue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in gym_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "yellow", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in cemetary_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "green", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in parks_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "purple", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in beaches_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "black", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in shopping_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "orange", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in grocery_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "lightblue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in resturant_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "gray", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in golf_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "cadetblue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in school_chi[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "beige", radius = 5, popup = popup)
    m.add_child(circle)
    
fig = folium.Figure(width = 900, height = 400)
fig.add_child(m)
m

**DALLAS METRO AREA**



In [ ]:
import folium


m = folium.Map(location = [32.7767,  -96.7970], zoom_start = 11)

cols = ["Type", "Longitude", "Latitude"]
for name, lat, lng in dalmetro_adj[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "red", radius = 10, popup = popup)
    m.add_child(circle)
for name, lng, lat in hospitals_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "blue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in gym_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "yellow", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in cemetary_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "green", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in parks_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "purple", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in beaches_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "black", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in shopping_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "orange", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in grocery_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "lightblue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in resturant_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "gray", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in golf_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "cadetblue", radius = 5, popup = popup)
    m.add_child(circle)
for name, lng, lat in school_dal[cols].itertuples(index = False):
    popup = folium.Popup(name, parse_html = True)
    circle = folium.Circle([float(lat), float(lng)], color = "beige", radius = 5, popup = popup)
    m.add_child(circle)
    
fig = folium.Figure(width = 900, height = 400)
fig.add_child(m)
m